# Testing Model1

Testing recommender systems is less intuitive than testing other predictive models. There are many common metrics to use, which can be found [here in this useful Medium article](https://medium.com/swlh/rank-aware-recsys-evaluation-metrics-5191bba16832). The idea of these methods are all similar: generate a list of top ranked items to show the user, and your score is based on how many of the items you recommend are relevant to the user. The word "relevant" is not well-defined, but normally means that the user has interacted with this item in the past. 

In our case, our items are the restaurants and the interactions are orders. One issue with our data is that between 30-40% of our users have only ordered from a single restaurant, which makes the above methods tricky/impossible to apply.

Recall that our model takes in a user's past five (5) orders as inputs together with one restaurant R and tries to predict how likely the user will order from R next. We will evaluate our model as follows. Given a user's sequence of five restaurants, and given the target restaurant R which is next in the sequence, we use the model to generate a ranked list of $k=5$ more restaurants that it thinks is the most likely to be ordered from next. If R is among the $k$ generated restaurants, we add $+1$ to the score, otherwise $+0$. 

Since there are only $100$ restaurants to recommend from, it should be very feasible to simply score each one, sort the rankings, and slice the top 5 as recommendations. This is a very privillaged position we are in since many recommender systems are deployed in a context where there are millions of items to recommend from. In that case, one could perform clustering based on the customer and vendor embeddings before ranking within the clusters.  

Each user in the test set may possibly geenerate many length $5+1$ sequences, and we shall use all of these during the evaluation process. In the deployed version of the model, we could simply give recommendations based on the last $5$ orders made by the user.

Baselines:
1) Recommend the $5$ most popular restaurants.
2) Given a sequence of $5$ orders, $m$ of which are unique and not null, recommend those restaurants together with the $5-m$ most popular restaurants. 

We will define 'popularity' of a vendor to be equal to its number of orders in the training data. Note that the unordered set of top five most popular vendors would be the same if we changed the definition of popularity of a vendor to be equal to its number of unique customers (although their rankings are slightly different, see the last cell of "Munging.ipynb"). 

In [1]:
import pandas as pd
import pickle
import torch
from PreprocessingHelpers import CustomDataset
from torch.utils.data import Dataset, DataLoader
from Models.Models import Model1, Model2, Model3


## Load Test Data

In [2]:
with open("ProcessedData/test_sequences_padded_dataset.pkl", "rb") as file:
    test_sequences_padded_dataset = pickle.load(file)

test_loader = DataLoader(test_sequences_padded_dataset, batch_size=1)
num_trials = test_sequences_padded_dataset.vendor.shape[0]

In [3]:
with open("ProcessedData/vendors_tensor.pkl", "rb") as file:
    vendors_tensor = pickle.load(file)

model1 = Model1(vendors=vendors_tensor, d_fc=64)
model2 = Model2(vendors=vendors_tensor, d_fc=64)
model3 = Model3(vendors=vendors_tensor, d_fc=64)

In [4]:
with open("ProcessedData/popular_vendors.pkl", "rb") as file:
    popular_vendors = pickle.load(file)

## Define Scoring for Model & Baselines

In [5]:
popular_vendors.head(10)

id
28    3237
25    2717
14    2681
19    2453
18    2184
15    2159
75    1377
21    1274
36    1145
6     1078
Name: num_orders, dtype: int64

In [6]:
k_most_popular = popular_vendors[:5].index.tolist()
k_most_popular

[28, 25, 14, 19, 18]

In [7]:
def baseline1_scoring(target:int, k_most_popular:list):
    if target in k_most_popular:
        return 1
    else:
        return 0

In [8]:
def baseline2_scoring(seq:list, target:int, k_most_popular:list):
    seq = list(set(seq))    # remove duplicates
    try:
        seq.remove(0)       # remove null-token
    except ValueError:
        pass
    m = len(seq)
    seq = seq + k_most_popular[:-m]
    if target in seq:
        return 1
    else:
        return 0

In [9]:
def model_scoring(seq:torch.tensor, target:int, model, k:int=5):
    seq = seq.view(1, -1)
    y = torch.ones([100, 1], dtype=torch.long)      # 100 vendors
    seq_dupe = y @ seq                              # 100 x 5 matrix
    v_ids = torch.arange(start=1, end=101, dtype=torch.long)
    rankings = model.forward(c_seq=seq_dupe, v_id=v_ids).view(-1)    
    top_k = torch.topk(rankings, k)[1][:5]          # Essentially argmax for top k
    top_k = top_k + 1                               # Shift indices to match vendors
    if target in top_k:
        return 1
    else:
        return 0

In [11]:
# Score baselines
print("SCORING\n=======================================================")
print("Random:\t\t2384 / 47677 = 5.00% ")
baseline1_score = 0
baseline2_score = 0
for i, (c_seq, v_id) in enumerate(test_loader):
    target = v_id.item()
    c_seq_list = c_seq.view(-1).tolist()
    baseline1_score += baseline1_scoring(target=target, k_most_popular=k_most_popular)
    baseline2_score += baseline2_scoring(seq=c_seq_list, target=target, k_most_popular=k_most_popular)
print(f'Baseline1:\t{baseline1_score} / {num_trials} = {baseline1_score*100/num_trials:.2f}%')
print(f'Baseline2:\t{baseline2_score} / {num_trials} = {baseline2_score*100/num_trials:.2f}%')
print("")

# Score model1 at different epochs
for epoch in range(0, 201, 10):
    PATH = "Models/model1_epoch"+str(epoch)+".pt"
    checkpoint = torch.load(PATH)
    model1.load_state_dict(checkpoint['model_state_dict'])
    model1.eval()
    model1_score = 0
    for i, (c_seq, v_id) in enumerate(test_loader):
        target = v_id.item()
        c_seq_list = c_seq.tolist()
        with torch.no_grad():
            model1_score += model_scoring(seq=c_seq, target=target, model=model1)
    print(f'Model1_{epoch}:\t{model1_score} / {num_trials} = {model1_score*100/num_trials:.2f}%')
print("")

# Score model2 at different epochs
for epoch in range(0, 201, 10):
    PATH = "Models/model2_epoch"+str(epoch)+".pt"
    checkpoint = torch.load(PATH)
    model2.load_state_dict(checkpoint['model_state_dict'])
    model2.eval()
    model2_score = 0
    for i, (c_seq, v_id) in enumerate(test_loader):
        target = v_id.item()
        c_seq_list = c_seq.tolist()
        with torch.no_grad():
            model2_score += model_scoring(seq=c_seq, target=target, model=model2)
    print(f'Model2_{epoch}:\t{model2_score} / {num_trials} = {model2_score*100/num_trials:.2f}%')
print("")

# Score model3 at different epochs
for epoch in range(0, 201, 10):
    PATH = "Models/model3_epoch"+str(epoch)+".pt"
    checkpoint = torch.load(PATH)
    model3.load_state_dict(checkpoint['model_state_dict'])
    model3.eval()
    model3_score = 0
    for i, (c_seq, v_id) in enumerate(test_loader):
        target = v_id.item()
        c_seq_list = c_seq.tolist()
        with torch.no_grad():
            model3_score += model_scoring(seq=c_seq, target=target, model=model3)
    print(f'Model3_{epoch}:\t{model3_score} / {num_trials} = {model3_score*100/num_trials:.2f}%')
print("Done!")

SCORING
Random:		2384 / 47677 = 5.00% 
Baseline1:	9233 / 47677 = 19.37%
Baseline2:	22134 / 47677 = 46.42%

Model1_0:	15579 / 47677 = 32.68%
Model1_10:	19627 / 47677 = 41.17%
Model1_20:	21271 / 47677 = 44.61%
Model1_30:	21525 / 47677 = 45.15%


KeyboardInterrupt: 

In [11]:
PATH = "Models/model3_epoch30.pt"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

seq = iter(test_loader).next()[0]
print(seq)
seq = seq.view(1, -1)
y = torch.ones([100, 1], dtype=torch.long)      # 100 vendors
seq_dupe = y @ seq                              # 100 x 5 matrix
v_ids = torch.arange(start=1, end=101, dtype=torch.long)
rankings = model.forward(c_seq=seq_dupe, v_id=v_ids).view(-1)    
top_k = torch.topk(rankings, 5)[1][:5]          # Essentially argmax for top k

print(rankings.sort())
print(len(rankings.tolist()))

tensor([[0, 0, 0, 0, 0]])
torch.return_types.sort(
values=tensor([-1.8947, -0.2750,  0.1777,  0.2211,  0.2352,  0.2771,  0.2915,  0.3192,
         0.4320,  0.4535,  0.4745,  0.4770,  0.5204,  0.5537,  0.5568,  0.6537,
         0.6652,  0.7893,  0.8370,  0.8842,  0.9073,  0.9184,  0.9256,  0.9333,
         0.9869,  1.0183,  1.0595,  1.0597,  1.1827,  1.1938,  1.2413,  1.3262,
         1.4613,  1.4752,  1.5227,  1.6915,  1.7520,  1.8084,  1.8130,  1.8162,
         1.8978,  1.9081,  2.0173,  2.0688,  2.1732,  2.2013,  2.2482,  2.2524,
         2.2655,  2.3086,  2.3252,  2.3374,  2.3715,  2.3715,  2.4481,  2.5075,
         2.5914,  2.6870,  2.7724,  2.8197,  2.8339,  2.8449,  3.0061,  3.2087,
         3.2184,  3.2267,  3.3115,  3.3319,  3.5376,  3.7694,  3.8482,  4.0569,
         4.1448,  4.2146,  4.3211,  4.4387,  4.4860,  4.5340,  4.5407,  4.5507,
         4.6005,  4.6885,  5.0842,  5.1168,  5.1998,  5.4003,  5.4051,  5.4095,
         5.4135,  5.4139,  5.4508,  5.5101,  5.5362,  5.5765, 